In [13]:
!pip install -q google-cloud-bigquery google-generativeai google-cloud-aiplatform scikit-learn

In [12]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.cloud import bigquery, aiplatform
import google.generativeai as genai
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import json

# INIT
PROJECT_ID = "qwiklabs-gcp-01-788c9d3b43e9"
REGION = "us-central1"

aiplatform.init(project=PROJECT_ID, location=REGION)
bq_client = bigquery.Client(project=PROJECT_ID)

genai.configure(api_key="YOUR_API_KEY")


In [30]:
from vertexai.language_models import TextEmbeddingModel
def get_embedding(text: str):
    model = TextEmbeddingModel.from_pretrained("text-embedding-005")
    embeddings = model.get_embeddings([text])
    return np.array(embeddings[0].values, dtype=np.float32)


In [31]:
model = genai.GenerativeModel(
    model_name="gemini-2.5-pro-preview-06-05",
    system_instruction=(
       "You are a helpful FAQ Assistant respond only to the questions."
    )
)

chat = model.start_chat()


In [32]:
def get_context_from_bigquery(user_input: str, top_k: int = 5):
    query = """
    SELECT question, answer, ml_generate_embedding_result
    FROM `AuroraBayFaqs.aurora_bay_faqs_embedded`
    """
    df = bq_client.query(query).to_dataframe()

    # Convert to vectors
    df["embedding_vector"] = df["ml_generate_embedding_result"].apply(lambda x: np.array(x, dtype=np.float32))

    # User input embedding
    user_vector = get_embedding(user_input).reshape(1, -1)

    # Cosine similarity
    similarities = cosine_similarity(user_vector, df["embedding_vector"].tolist())[0]
    df["score"] = similarities

    top = df.sort_values("score", ascending=False).head(top_k)
    return "\n".join([f"Q: {row.question}\nA: {row.answer}" for _, row in top.iterrows()])


In [33]:
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    try:
        context = get_context_from_bigquery(user_input)
        if not context:
            context = "No relevant information found."

        prompt = f"Context:\n{context}\n\nQuestion:\n{user_input}"
        response = chat.send_message(prompt)
        print("Gemini:", " ".join(response.text.split()[:20]))
    except Exception as e:
        print("Error:", e)


You: When was Aurora Bay founded?
Gemini: Aurora Bay was founded in 1901 by a group of fur traders who recognized the region’s strategic coastal location.
You: Who is the current mayor of Aurora Bay?
Gemini: The current mayor is Linda Greenwood, elected in 2021 for a four-year term.
You: exit
